# Anisotropic Mg tracer calculation

In [1]:
import numpy as np
from onsager import OnsagerCalc
from onsager import crystal
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.constants import physical_constants
kB = physical_constants['Boltzmann constant in eV/K'][0]
#print (kB)

In [2]:
a=3.18957309719204
c_a=1.6264299717255
HCP = crystal.Crystal.HCP(a0=a,c_a=c_a, chemistry="Mg")
print(HCP)

#Lattice:
  a1 = [ 1.59478655 -2.76225133  0.        ]
  a2 = [ 1.59478655  2.76225133  0.        ]
  a3 = [ 0.          0.          5.18761728]
#Basis:
  (Mg) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Mg) 0.1 = [ 0.66666667  0.33333333  0.75      ]


In [3]:
sitelist = HCP.sitelist(0)
#print (HCP.sitelist(0))
#print(sitelist)
vacancyjumps = HCP.jumpnetwork(0, 1.01*a)
for jlist in vacancyjumps:
    print('---')
    for (i,j), dx in jlist:
        print(i, '-', j, dx,np.linalg.norm(dx))
#sitelist

---
1 - 1 [  3.18957310e+00   1.11022302e-16   0.00000000e+00] 3.18957309719
1 - 1 [ -3.18957310e+00  -1.11022302e-16  -0.00000000e+00] 3.18957309719
1 - 1 [ 1.59478655 -2.76225133  0.        ] 3.18957309719
1 - 1 [-1.59478655  2.76225133 -0.        ] 3.18957309719
0 - 0 [ -3.18957310e+00   1.11022302e-16   0.00000000e+00] 3.18957309719
0 - 0 [  3.18957310e+00  -1.11022302e-16  -0.00000000e+00] 3.18957309719
1 - 1 [-1.59478655 -2.76225133  0.        ] 3.18957309719
1 - 1 [ 1.59478655  2.76225133 -0.        ] 3.18957309719
0 - 0 [-1.59478655 -2.76225133  0.        ] 3.18957309719
0 - 0 [ 1.59478655  2.76225133 -0.        ] 3.18957309719
0 - 0 [-1.59478655  2.76225133  0.        ] 3.18957309719
0 - 0 [ 1.59478655 -2.76225133 -0.        ] 3.18957309719
---
1 - 0 [ 1.59478655 -0.92075044 -2.59380864] 3.18103265953
0 - 1 [-1.59478655  0.92075044  2.59380864] 3.18103265953
1 - 0 [ 1.59478655 -0.92075044  2.59380864] 3.18103265953
0 - 1 [-1.59478655  0.92075044 -2.59380864] 3.18103265953
1 - 

In [4]:
HCPdiffuser = OnsagerCalc.VacancyMediated(HCP, 0, sitelist, vacancyjumps, 1)
#print (HCPdiffuser)
#HCPdiffuser.omegalist()

In [5]:
len(HCPdiffuser.interactlist())
for state in HCPdiffuser.interactlist():
    print(state)

1.[0,0,0]:0.[1,0,1] (dx=[1.5947865485960198,-0.9207504431319062,2.5938086411412327])
1.[0,0,0]:1.[1,0,0] (dx=[1.5947865485960198,-2.7622513293957187,0.0])


We need to specify the vacancy prefactor and formation energy, though the $L_{ij}$ needs to be explicitely multiplied by $c_\text{v}$, so it is *strictly used as a reference for the transition state energies*.

In [6]:
#HCPtracer = {'preV': np.array([1.0/1.1075157070506694]), 'eneV': np.array([0.0]), 
#             'preT0': np.array([129.2186479/36.7840204101001, 129.2186479/34.5772672908345]), 'eneT0': np.array([1.211, 1.23])}
HCPtracer = {'preV': np.array([1.0]), 'eneV': np.array([0.814]), 
             'preT0': np.array([129.2186479/36.7840204101001, 129.2186479/34.5772672908345]), 'eneT0': np.array([1.211, 1.23])}
HCPtracer.update(HCPdiffuser.maketracerpreene(**HCPtracer))
#for k,v in zip(HCPtracer.keys(), HCPtracer.values()): print(k,v)

Anisotropic diffusivity from Onsager calculation; needs to be multiplied by $c_\text{v}/k_\text{B}T$ to make $L$ for the vacancy or $c_\text{v}c_\text{s}/k_\text{B}T$ for the solute (or solute/vacancy). To get the diffusivity of the tracer, we will need to multiply by $c_\text{v}$ only.

In [7]:
Temp=np.arange(100, 923, 100)
D_Onsager=[]
for T in Temp:
    pre=1e-8 # THz and Angstrom unit scaling
    Lvv, Lss, Lsv, L1vv = HCPdiffuser.Lij(*HCPdiffuser.preene2betafree(kB*T, **HCPtracer))
    Lss=Lss*pre
    D_Onsager.append([Lss[0,0],Lss[1,1],Lss[2,2]])
    print(T, Lss[0,0],Lss[1,1],Lss[2,2])
D_Onsager=np.array(D_Onsager)

100 3.56079795643e-27 3.56079795643e-27 7.52603965086e-28
200 4.28889205421e-17 4.28889205421e-17 2.12416600644e-17
300 1.0031899208e-13 1.0031899208e-13 6.42728964174e-14
400 4.87214647009e-12 4.87214647009e-12 3.53289274016e-12
500 5.01349931034e-11 5.01349931034e-11 3.90957584172e-11
600 2.37332231418e-10 2.37332231418e-10 1.94139246641e-10
700 7.20757760426e-10 7.20757760426e-10 6.09874127154e-10
800 1.65840136534e-09 1.65840136534e-09 1.43907792211e-09
900 3.17112544026e-09 3.17112544026e-09 2.80591255474e-09


# Analytical tracer diffusion result

We take the tabular data directly from Koiwa and Ishioka's paper (doi://10.1080/01418618308245263) in tabular form, put it here and use spline interpolation instead of the polynomial ratios. This is mainly to get the exact values when the particular ratio is calculated.

In [8]:
#Correlations from Koiwa and Ishioka’s
# def fz(x):
#     return((1+8.77*x+7.37*x**2+0.65*x**3)/(1+9.90*x+10.9*x**2+x**3))
# def fx(x):
#     return((0.56+4.38*x+3.67*x**2+0.64*x**3)/(1+5.75*x+4.11*x**2+x**3))
from scipy import interpolate
# nuB / nuA
nuBA = np.arange(0.0, 1.0001, 0.1)
fxBAraw = np.array([0.560057, 0.642376, 0.683451, 0.711016, 0.730851, 0.745626, 
                    0.756849, 0.765469, 0.772123, 0.777260, 0.781205])
fzBAraw = np.array([1.000000, 0.929038, 0.892655, 0.866949, 0.847185, 0.831277, 
                    0.818082, 0.806901, 0.797270, 0.788865, 0.781451])
fxBA = interpolate.UnivariateSpline(nuBA, fxBAraw, s=0)
fzBA = interpolate.UnivariateSpline(nuBA, fzBAraw, s=0)
# nuA / nuB
nuAB = np.arange(1.0, -0.0001, -0.1)
fxABraw = np.array([0.781205, 0.784485, 0.787315, 0.789404, 0.790295, 0.789251, 
                    0.785032, 0.775451, 0.756394, 0.719387, 0.644545])
fzABraw = np.array([0.781451, 0.774166, 0.766204, 0.757451, 0.747762, 0.736951, 
                    0.724773, 0.710893, 0.694845, 0.675941, 0.653109])
fxAB = interpolate.UnivariateSpline(nuAB[::-1], fxABraw[::-1], s=0)
fzAB = interpolate.UnivariateSpline(nuAB[::-1], fzABraw[::-1], s=0)
def fx(x):
    if x <= 1: return fxBA(x)
    else: return fxAB(1/x)
def fz(x):
    if x <= 1: return fzBA(x)
    else: return fzAB(1/x)

In [9]:
def Basal_rate(T):
    prefactor=(129.2186479/36.78402041)
    mb=0.397
    return(prefactor*np.exp(-mb/(kB*T)))
def c_rate(T):
    prefactor=(129.2186479/34.57726729)
    mc=0.416
    return(prefactor*np.exp(-mc/(kB*T)))

In [10]:
def Vconc(T):
    entropy_contribution=1.1075157070506694
    formationE=0.814228281
    prob=entropy_contribution*np.exp(-formationE/(kB*T))
    return(prob)

In [11]:
def AnalyticalD(T):
    wb=Basal_rate(T)
    wc=c_rate(T)
    x=wc/wb  # NOTE: this was reversed in the code Ravi sent; I'm putting it back to the definition in K&S 1983 paper
    c=a*c_a
    Dxx=0.5*Vconc(T)*a**2*fx(x)*(3*wb+wc)
    Dzz=0.75*Vconc(T)*c**2*fz(x)*wc
    return(np.array([Dxx,Dxx,Dzz]))

In [12]:
D_Analytical=[]
vac = np.array([Vconc(T) for T in Temp])  # equilibrium vacancy concentration: we'll need this to scale D_Onsager
for T in Temp:
    D=AnalyticalD(T)*1E-8 #Unit scaling
    print(T, D[0],D[1],D[2])
    D_Analytical.append([D[0],D[1],D[2]])
D_Analytical=np.array(D_Analytical)

100 3.63847485635e-68 3.63847485635e-68 7.68058421208e-69
200 1.44236638224e-37 1.44236638224e-37 7.14270905373e-38
300 2.3296989804e-27 2.3296989804e-27 1.49259710608e-27
400 2.9733358607e-22 2.9733358607e-22 2.15602919882e-22
500 3.44691069356e-19 3.44691069356e-19 2.68793529085e-19
600 3.80618149763e-17 3.80618149763e-17 3.11347801137e-17
700 1.09635937713e-15 1.09635937713e-15 9.27692209715e-16
800 1.36340995398e-14 1.36340995398e-14 1.18309989441e-14
900 9.68468074552e-14 9.68468074552e-14 8.56931026765e-14


Comparison of Onsager and analyitcal result, printing ratio of Onsager D/ Analytical D. We need to *now* multiply by the equilibrium vacancy concentration for the Onsager calculated version (since it gets multiplied by $c_\text{v}$). Now the agreement is excellent, with only "significant" deviation only happening at the smallest temperatures, where the diffusivity becomes very anisotropic (anisotropy ratio around 10, see a difference of $<10^{-3}$, which would be improved with denser $k$-point meshing).

In [13]:
D_Onsager_scaled = np.array([v*D for v,D in zip(vac, D_Onsager)])
Ratio=D_Onsager_scaled/(D_Analytical)
for i,T in enumerate(Temp):
    print (T, Ratio[i][0],Ratio[i][1],Ratio[i][2], c_rate(T)/Basal_rate(T))

100 0.999250692702 0.999250692702 1.00050381936 0.11730339367
200 0.999925682932 0.999925682932 1.0000545574 0.353256004263
300 0.999997193825 0.999997193825 1.00000162821 0.510131972868
400 1.00000095271 1.00000095271 0.999999679534 0.613026205459
500 1.00000033237 1.00000033237 1.0000001134 0.684473971015
600 0.999999749657 0.999999749657 1.00000045963 0.736674328539
700 1.00000020101 1.00000020101 0.999999989741 0.776380349624
800 1.00000036082 1.00000036082 0.999999689747 0.807558111738
900 0.999999660796 0.999999660796 1.00000001029 0.832670540646


KMC result for three temperatures, statisical error in KMC values is ~0.05%

In [14]:
# 700K, 800K, 900K
D_KMC=np.array([[1.10228325e-15,   1.10133965e-15, 9.30127592e-16],
                [1.36918534e-14,   1.37001681e-14,     1.18546593e-14],
                [9.73279952e-14,   9.73412524e-14,  8.58637944e-14]])

Comparison of Onsager (which matches the analytic result to $10^{-7}$) and KMC result, printing ratio of Onsager D/ KMC D. Agreement is $\sim 5\times10^{-3}$.

In [15]:
Ratio=D_Onsager_scaled[-3:]/D_KMC
for i,T in enumerate(Temp[-3:]):
    print (T, Ratio[i][0],Ratio[i][1],Ratio[i][2])

700 0.994626016055 0.995478186508 0.997381658362
800 0.995782240792 0.995177895609 0.998003820618
900 0.99505568162 0.994920161972 0.998012074321
